In [10]:
import os
import pandas as pd
import openai
from google.cloud import vision
import io
import time

In [11]:

# Set up your OpenAI API key
openai.api_key = 'sk-proj-hIOossFdrZSyM5ixG4vPT3BlbkFJv9bTi0HWRujvSlWCnsIW'


In [12]:

# Set up your Google Cloud Vision API client
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/njlalwani/Downloads/fiery-rarity-430115-b5-5cad704232f1.json'
client = vision.ImageAnnotatorClient()


I0000 00:00:1721846436.377739 4881819 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [13]:
# Extract Lables from Vision API

def get_image_labels(image_path):
    """Use Google Vision API to get labels from the image."""
    try:
        with io.open(image_path, 'rb') as image_file:
            content = image_file.read()

        image = vision.Image(content=content)
        response = client.label_detection(image=image)
        labels = response.label_annotations

        return [label.description for label in labels]
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return []


In [14]:

def generate_prompt_from_labels(labels, code, color):
    """Generate a prompt for GPT-4 based on image labels."""
    prompt = (
        f"Product Code: {code}\n"
        f"Product Color: {color}\n"
        f"Image Labels: {', '.join(labels)}\n\n"
        "Generate the following:\n"
        "- Long Description(information rich description to display on the website aronud 50 words\n"
        "- Short Description(small 20 word description in artsy style to display on the catalogue)\n"
        "- Keywords(precise things that one might type to find this in a search)\n"
        "- Tags(one word of info for filtering purposes[colour,code,long/short etc])\n\n"
        "Try not to use ':' in the sentences but you can use it just as a separator."
    )
    return prompt


In [15]:

def generate_product_info(prompt):
    """Generate product information using GPT-4."""
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=250
    )

    response_text = response.choices[0].message['content'].strip()
    print("API Response:", response_text)  # Debugging line

    long_description = short_description = keywords = tags = "N/A"
    
    try:
        response_lines = response_text.split('\n')
        
        for line in response_lines:
            if 'Long Description' in line:
                long_description = line.split(':: ', 1)[1] if ':: ' in line else "N/A"
            elif 'Short Description' in line:
                short_description = line.split(':: ', 1)[1] if ':: ' in line else "N/A"
            elif 'Keywords' in line:
                keywords = line.split(':: ', 1)[1] if ':: ' in line else "N/A"
            elif 'Tags' in line:
                tags = line.split(':: ', 1)[1] if ':: ' in line else "N/A"
    except Exception as e:
        print(f"Error parsing response: {e}")
    
    return long_description, short_description, keywords, tags


In [16]:

def process_images(image_directory, output_csv, batch_size=10):
    """Process images in the directory and save product information to a CSV file."""
    # Load existing data if the CSV exists
    if os.path.exists(output_csv):
        df = pd.read_csv(output_csv)
    else:
        df = pd.DataFrame(columns=['Filename', 'Long Description', 'Short Description', 'Keywords', 'Tags'])

    # Get list of image files with num=1 in filename
    image_files = [f for f in os.listdir(image_directory) if f.endswith(('.jpg', '.png')) and '_1.' in f]

    for i in range(0, len(image_files), batch_size):
        batch_files = image_files[i:i + batch_size]
        new_data = []
        
        for image_file in batch_files:
            image_path = os.path.join(image_directory, image_file)
            code, color, num_ext = image_file.split('_')[0], image_file.split('_')[1], image_file.split('_')[2]
            num, ext = num_ext.split('.')

            # Extract labels
            labels = get_image_labels(image_path)

            # Generate GPT-4 prompt and get product info
            prompt = generate_prompt_from_labels(labels, code, color)
            long_description, short_description, keywords, tags = generate_product_info(prompt)

            # Add to new_data list
            new_data.append({
                'Filename': image_file,
                'Long Description': long_description,
                'Short Description': short_description,
                'Keywords': keywords,
                'Tags': tags
            })
        
        # Concatenate new data to the existing DataFrame
        df = pd.concat([df, pd.DataFrame(new_data)], ignore_index=True)

        # Save intermediate results to CSV
        df.to_csv(output_csv, index=False)
        
        print(f"Processed batch {i // batch_size + 1} / {len(image_files) // batch_size + 1}")

        # Pause mechanism
        user_input = input("Press Enter to continue to the next batch or type 'pause' to pause: ")
        if user_input.lower() == 'pause':
            print("Pausing... Type 'resume' to continue.")
            while input().lower() != 'resume':
                time.sleep(1)


In [17]:

# Directory containing your images
image_directory = 'test_photos'
output_csv = 'product_info.csv'
batch_size = 1

process_images(image_directory, output_csv, batch_size)



API Response: Long Description: Embrace elegance with our sleek E1544 dress. Featured in a classic black shade, this one-piece sheath dress enhances your silhouette perfectly. Its chic design boasts a neat waistline, artful neck, and streamlined sleeves that add an air of sophistication for day or night.

Short Description: Classic black E1544 sheath dress - Designed for a graceful silhouette. Perfect fusion of style and elegance. 

Keywords: black sheath dress, one-piece outfit, E1544, day dress, waist-fit, sleeve dress, elegant arm style, human body dress, stylish neck

Tags: black, E1544, Long, sheath-dress, one-piece, day-dress, sleeve, waist, neck
Processed batch 1 / 4
API Response: Long Description: This ravishing red sheath dress, product code E1544, is designed with impeccable attention to modern tailoring. Perfect for daytime affairs, it's sculpted to gracefully adorn the human body. With a flattering neck, shoulder, and arm details, it's a single-piece garment that personifie